In [3]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

# import _utils # amanda's collected utils

import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils
import datetime as dt

import gcsfs

from calitp_data.storage import get_fs
fs = get_fs()

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"

ModuleNotFoundError: No module named 'calitp'

# Spatially Join SWITRS Data to Projects

In [ ]:
# load projects
with get_fs().open(f'{GCS_FILE_PATH}nonshopp_gdf.geojson') as f:
    all_proj = gpd.read_file(f)

In [ ]:
all_proj.head()

In [ ]:
# buffer the projects w/ geometry - start w/ 100ft?
all_proj_geom = (all_proj
                   >> filter(_.geometry.is_valid)
                   )

In [ ]:
# re-project
all_proj_geom = all_proj_geom.to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [ ]:
all_proj_geom.explore(tiles="cartodbpositron")

In [ ]:
# add buffer - 100 ft ~ 30m
all_proj_geom['b100'] = all_proj_geom.buffer(30)
all_proj_geom['b200'] = all_proj_geom.buffer(61)

In [ ]:
all_proj_geom = all_proj_geom.set_geometry('b100')

In [ ]:
# load safety data
switrs = gpd.read_parquet(f'{GCS_FILE_PATH}SWITRS_clean.parquet')

In [ ]:
# drop dt for folium mapping
switrs = (switrs >> select(-_.collision_dt))

In [ ]:
# test mapping crashes only
switrs.head(100).explore(tiles="cartodbpositron")

In [ ]:
# overlay all crashes w/ projects
m2 = switrs.explore(m=m
)

In [ ]:
m2

In [ ]:
# clip crashes to those within buffer
switrs_clip = switrs.clip(all_proj_geom)

In [ ]:
switrs_clip.plot()

In [ ]:
# spatial join - left side is safety data. Aggregate up fatals, severely injured, etc to each project

Test Metrics:
* n fatalities within 100ft buffer of project area over 5 year lookback period
    * test different temporal study periods; rural may need more time
    * test different buffers; different project types may need different sizes
* n fatalities+severely injured (KSIs) w/in buffer area
* n ped/bike crashes w/in buffer area
* % ped/bike crashes out of all crashes w/in buffer area